In [12]:
import scipy
import numpy as np
import seaborn as sns
import fiona
import rasterio
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.utils.data as torch_data
import sys
from shapely import geometry
from rasterio.mask import mask
from tqdm import tqdm_notebook
from extractor_helper import extractor
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score


In [13]:
import torch


In [14]:
torch.__version__

'1.4.0'

In [15]:
torch.cuda.current_device()

0

Data Import

In [16]:
sat_image = rasterio.open("project/1030010056130F00_MS_Pan_modified_KNN.tif")
sat_points1 = fiona.open("project/Points_MS_Pan_KNN.shp", "r")
sat_points2 = fiona.open("project/AB_points.shp", "r")

test2_img = rasterio.open("Eval/test2/pp_2_sat_modified.tif")
test2_points = fiona.open("Eval/test2/points_2_modified_Copy.shp", "r")

test3_img = rasterio.open("Eval/test3/pp_3_sat_modified.tif")
test3_points = fiona.open("Eval/test3/targets_Copy.shp", "r")

test4_img = rasterio.open("Eval/test4/pp_4_sat_modified_spline.tif")
test4_points = fiona.open("Eval/test4/modified_points_Copy.shp", "r")

In [17]:
class TreesData():
  def __init__(self, X, y):
    super().__init__()
    self.X = torch.tensor(X, dtype=torch.float32)
    self.y = torch.tensor(y, dtype=torch.float32)
    
  def __len__(self):
    return self.X.shape[0]
  
  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]
    

In [18]:
class AutoEncoder(nn.Module):
  def __init__(self,encoder,decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    
  def forward(self,x):
    z = self.encoder(x)
    out = self.decoder(z)
    return out

In [19]:
def train(epochs, net, criterion, optimizer,
          train_loader, val_loader, scheduler=None,
          verbose=True, save_dir=None):
  loss_tr =0
  loss_val =0
  net.to(device)

  for epoch in range(1,epochs+1):
      net.train()
      for X,_ in train_loader:
          X = X.to(device)
          pred = net(X)
          loss = criterion(pred,X)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
    
      net.eval()
      for X,_ in val_loader:
          X = X.to(device)
          val_pred = net(X)
          val_loss = criterion(val_pred,X)
#       loss /= len(train_loader)
#       val_loss /= len(val_loader)
      loss_tr +=loss.item()
      loss_val +=val_loss.item()

  
      if scheduler is not None:
          schduler.step()
    
      freq = max(epochs//20,1)
      if verbose and epoch%freq==0:
        print('Epoch {}/{} || Loss:  Train {:.6f} | Validation {:.6f}'.format(epoch, epochs, loss.item(), val_loss.item())) 
  return (loss_tr/epochs), (loss_val/epochs)

In [20]:
encoder = lambda hid,red_size: nn.Sequential(
  
  nn.Conv2d(8,60,3),
  nn.BatchNorm2d(60),
  nn.ReLU(),

  nn.Conv2d(60,200,3),
  nn.BatchNorm2d(200),
  nn.ReLU(),

  nn.Flatten(),
  nn.Linear(200*red_size*red_size,300),
  nn.BatchNorm1d(300),
  nn.ReLU(),

  nn.Linear(300,hid),
  nn.BatchNorm1d(hid),
)

decoder = lambda hid,orginal_size: nn.Sequential(
  nn.Linear(hid,600),
  nn.BatchNorm1d(600),
  nn.ReLU(),

  nn.Linear(600,1000),
  nn.BatchNorm1d(1000),
  nn.ReLU(),

  nn.Linear(1000,orginal_size*orginal_size*8),
  nn.Sigmoid(),
  Unflatten(),
)


In [21]:
class Unflatten(nn.Module):
  def forward(self, input):
    return input.reshape(input.shape[0],8,orginal_size,orginal_size)

In [22]:
import pandas as pd 

b = ['AdaBoostClassifier_acc_tr', 'AdaBoostClassifier_acc_ts','AdaBoostClassifier_f1_tr', 'AdaBoostClassifier_f1_ts','Ada_params',
     'XGBClassifier_acc_tr', 'XGBClassifier_acc_ts','XGBClassifier_f1_tr', 'XGBClassifier_f1_ts','XGB_params',
     'CatBoostClassifier_acc_tr', 'CatBoostClassifier_acc_ts','CatBoostClassifier_f1_tr', 'CatBoostClassifier_f1_ts','Cat_params',
     'RandomForestClassifier_acc_tr', 'RandomForestClassifier_acc_ts','RandomForestClassifier_f1_tr', 'RandomForestClassifier_f1_ts','RF_params',
     'SVC_acc_tr', 'SVC_acc_ts','SVC_f1_tr', 'SVC_f1_ts','SVC_params',
     'LogisticRegression_acc_tr', 'LogisticRegression_acc_ts','LogisticRegression_f1_tr', 'LogisticRegression_f1_ts','LG_params',
     'tr_loss','val_loss']

d = [0]*32


In [23]:
import sklearn
sklearn.__version__

'0.22.2.post1'

In [25]:
from tqdm import tqdm
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

for cut_size in tqdm((range(3,16,2))):
    results = pd.DataFrame(index=b,data=d)
    for bot_neck in tqdm((range(40,130,20))):

        patch2,coordinates2,labels2, = extractor(test2_img,test2_points,size=cut_size, normalize=True,labeling=True)
        patch3,coordinates3,labels3, = extractor(test3_img,test3_points,size=cut_size, normalize=True,labeling=True)
        patch4,coordinates4,labels4, = extractor(test4_img,test4_points,size=cut_size, normalize=True,labeling=True)

        patch2,patch3, patch4 = np.array(patch2), np.array(patch3), np.array(patch4)
        labels2, labels3, labels4 = np.array(labels2), np.array(labels3), np.array(labels4)
        patch2, patch3, patch4 = np.moveaxis(patch2,3,1), np.moveaxis(patch3,3,1), np.moveaxis(patch4,3,1)

        patch = np.concatenate((patch2, patch3, patch4),axis=0)
        patch = torch.tensor(patch, dtype=torch.float32).to(device)
        labels = np.concatenate((labels2,labels3,labels4),axis=0)

#         sm_images1, sm_points1,_ = extractor(sat_image,sat_points1,size=cut_size, normalize=True)
#         sm_images2,sm_point2,_= extractor(sat_image,sat_points2,size=cut_size, normalize=True)
#         data = np.array(sm_images1+sm_images2)
#         data = np.moveaxis(data,3,1)
#         targets = np.zeros(shape = data.shape[0])

#         X_Train, X_Test, Y_Train, Y_Test = train_test_split(data, targets,test_size=.20, shuffle=True,random_state=12)
#         train_data = TreesData(X_Train,Y_Train)
#         test_data = TreesData(X_Test,Y_Test)

#         orginal_size = X_Test.shape[-1]
#         side = lambda x: int(((x - 3)/1)+1)
#         red_size  = side(side(orginal_size))

#         del X_Train, X_Test,Y_Train,Y_Test,data,targets

#         net = AutoEncoder(encoder(bot_neck,red_size), decoder(bot_neck,orginal_size))
#         criterion = nn.MSELoss()
#         optimizer = torch.optim.Adam(net.parameters())
#         scheduler = None 
#         train_loader = torch_data.DataLoader(train_data,batch_size=300, shuffle=True)
#         val_loader = torch_data.DataLoader(test_data,batch_size=300,shuffle=True)
        
#         lsstr, lssts = train(200, net, criterion, optimizer, train_loader, val_loader,scheduler)
#         print(lsstr, lssts)

        net = torch.load(f'models/net_cutsize{cut_size}butneck{bot_neck}.pth')
        net.eval()
        embeddings_res = net.encoder(patch)
        embeddings_res = embeddings_res.cpu().detach().numpy()
        ros = SMOTE(random_state=12)
#         ros = RandomOverSampler(random_state=12)
        print(np.bincount(labels))
        x_train,x_test,y_train,y_test = train_test_split(embeddings_res,
                                                         labels,test_size=.2,
                                                         shuffle=True,
                                                         random_state=12,
                                                         stratify=labels)
       
        
        x_train, y_train = ros.fit_resample(x_train, y_train)
        print(np.bincount(y_train))
        

        clf1 = AdaBoostClassifier(n_estimators=100, random_state=12)
        clf2 = xgb.XGBClassifier(n_estimators=100, n_jobs=-1, random_state=12, ) 
        clf3 = CatBoostClassifier(iterations=1000, learning_rate=0.1,verbose=False)
        clf4 = RandomForestClassifier(n_jobs=-1, random_state=12)
        clf5 = SVC()
        clf6 = LogisticRegression(n_jobs=-1)

        parameters = [{"n_estimators":[20,30,50,100,150,200]},
                      {"n_estimators":[50,100,150,200,300,400,500], "max_depth":[3,5,6,10]},
                      {},
                      {"n_estimators":[50,100,150,200,300,400,500], "max_depth":[3,5,6,10,15,20]},
                      {'kernel':['rbf','poly','sigmoid'],'gamma':['scale','auto'],'decision_function_shape':['ovo', 'ovr']},
                      {}]

        clfs = {"Adaboost":clf1, "XGboost":clf2,'Catboost':clf3,"Random Forest":clf4, "SVM": clf5, "LogisticRegression": clf6}

        colm = []
        for i,names in enumerate(clfs):
              clf = GridSearchCV(clfs[names],param_grid=parameters[i], cv=5, n_jobs=-1, scoring='f1_macro',).fit(x_train,y_train)
              pred_train = clf.predict(x_train)
              pred_test = clf.predict(x_test)
              print(f'{names}: best params {clf.best_params_}\n')
              print('Train accuracy:\t {:.3}'.format(accuracy_score(y_train,pred_train)))
              print('Test accuracy:\t {:.3}'.format(accuracy_score(y_test,pred_test)))
              colm.append(accuracy_score(y_train,pred_train))
              colm.append(accuracy_score(y_test,pred_test))
              print('Train precision: {:.3}'.format(precision_score(y_train,pred_train,average='macro')))
              print('Test precision:\t {:.3}'.format(precision_score(y_test,pred_test,average='macro')))

              print('Train recall:\t {:.3}'.format(recall_score(y_train,pred_train,average='macro')))
              print('Test recall:\t {:.3}'.format(recall_score(y_test,pred_test,average='macro')))

              print('Train f1_score:\t {:.3}'.format(f1_score(y_train,pred_train,average='macro')))
              print('Test f1_score:\t {:.3}'.format(f1_score(y_test,pred_test,average='macro')),'\n',"-"*30)
              colm.append(f1_score(y_train,pred_train,average='macro'))
              colm.append(f1_score(y_test,pred_test,average='macro'))
              clfs[names]=clf
              colm.append(clf.best_params_)
        colm.append(0)
        colm.append(0)
        index = len(results.columns)
        results.insert(index,f'cut_size {cut_size}/bot_neck {bot_neck}',colm)
        results.to_csv(f'results{cut_size}.csv')




  0%|          | 0/7 [00:00<?, ?it/s]


  0%|          | 0/5 [00:00<?, ?it/s]

[518 315  42 137]
[414 414 414 414]
Adaboost: best params {'n_estimators': 200}

Train accuracy:	 0.78
Test accuracy:	 0.685
Train precision: 0.799
Test precision:	 0.575
Train recall:	 0.78
Test recall:	 0.567
Train f1_score:	 0.785
Test f1_score:	 0.57 
 ------------------------------
XGboost: best params {'max_depth': 10, 'n_estimators': 400}

Train accuracy:	 0.999
Test accuracy:	 0.714
Train precision: 0.999
Test precision:	 0.563
Train recall:	 0.999
Test recall:	 0.582
Train f1_score:	 0.999
Test f1_score:	 0.572 
 ------------------------------
Catboost: best params {}

Train accuracy:	 0.999
Test accuracy:	 0.749
Train precision: 0.999
Test precision:	 0.574
Train recall:	 0.999
Test recall:	 0.591
Train f1_score:	 0.999
Test f1_score:	 0.582 
 ------------------------------
Random Forest: best params {'max_depth': 15, 'n_estimators': 500}

Train accuracy:	 0.999
Test accuracy:	 0.764
Train precision: 0.999
Test precision:	 0.584
Train recall:	 0.999
Test recall:	 0.598
Train 




 20%|██        | 1/5 [01:50<07:21, 110.35s/it]

LogisticRegression: best params {}

Train accuracy:	 0.807
Test accuracy:	 0.65
Train precision: 0.805
Test precision:	 0.583
Train recall:	 0.807
Test recall:	 0.678
Train f1_score:	 0.805
Test f1_score:	 0.593 
 ------------------------------
[518 315  42 137]
[414 414 414 414]
Adaboost: best params {'n_estimators': 150}

Train accuracy:	 0.748
Test accuracy:	 0.64
Train precision: 0.76
Test precision:	 0.57
Train recall:	 0.748
Test recall:	 0.578
Train f1_score:	 0.752
Test f1_score:	 0.564 
 ------------------------------
XGboost: best params {'max_depth': 5, 'n_estimators': 150}

Train accuracy:	 0.999
Test accuracy:	 0.709
Train precision: 0.999
Test precision:	 0.579
Train recall:	 0.999
Test recall:	 0.596
Train f1_score:	 0.999
Test f1_score:	 0.586 
 ------------------------------
Catboost: best params {}

Train accuracy:	 0.999
Test accuracy:	 0.719
Train precision: 0.999
Test precision:	 0.596
Train recall:	 0.999
Test recall:	 0.614
Train f1_score:	 0.999
Test f1_score:	 




 40%|████      | 2/5 [04:05<05:53, 117.93s/it]

...

LogisticRegression: best params {}

Train accuracy:	 0.873
Test accuracy:	 0.581
Train precision: 0.869
Test precision:	 0.514
Train recall:	 0.873
Test recall:	 0.546
Train f1_score:	 0.87
Test f1_score:	 0.515 
 ------------------------------
[518 315  42 137]
[414 414 414 414]
Adaboost: best params {'n_estimators': 200}

Train accuracy:	 0.742
Test accuracy:	 0.532
Train precision: 0.752
Test precision:	 0.471
Train recall:	 0.742
Test recall:	 0.449
Train f1_score:	 0.744
Test f1_score:	 0.45 
 ------------------------------
XGboost: best params {'max_depth': 6, 'n_estimators': 100}

Train accuracy:	 0.999
Test accuracy:	 0.581
Train precision: 0.999
Test precision:	 0.434
Train recall:	 0.999
Test recall:	 0.432
Train f1_score:	 0.999
Test f1_score:	 0.432 
 ------------------------------
Catboost: best params {}

Train accuracy:	 0.999
Test accuracy:	 0.66
Train precision: 0.999
Test precision:	 0.509
Train recall:	 0.999
Test recall:	 0.509
Train f1_score:	 0.999
Test f1_score:	

/trinity/home/a.shtanchaev/.conda/envs/newone/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM: best params {'decision_function_shape': 'ovo', 'gamma': 'scale', 'kernel': 'rbf'}

Train accuracy:	 0.995
Test accuracy:	 0.69
Train precision: 0.995
Test precision:	 0.531
Train recall:	 0.995
Test recall:	 0.529
Train f1_score:	 0.995
Test f1_score:	 0.525 
 ------------------------------


/trinity/home/a.shtanchaev/.conda/envs/newone/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



100%|██████████| 5/5 [14:28<00:00, 173.74s/it]


100%|██████████| 7/7 [1:37:06<00:00, 832.36s/it]

LogisticRegression: best params {}

Train accuracy:	 0.908
Test accuracy:	 0.611
Train precision: 0.906
Test precision:	 0.524
Train recall:	 0.908
Test recall:	 0.57
Train f1_score:	 0.907
Test f1_score:	 0.535 
 ------------------------------
